In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from statsmodels.api import OLS


FILEPATH = "../Data/en.openfoodfacts.org.products.tsv"
df = pd.read_csv(FILEPATH, sep="\t", low_memory=False)

score_col = "nutrition-score-uk_100g"

# Parse OpenFoodFacts Dataset

In [2]:
# Drop rows with missing serving size or score
df.dropna(subset=[score_col, "serving_size"], inplace=True)

In [3]:
def filter_nutrients(column_name):
    return column_name.endswith("_100g") and column_name[0] != "-"

# Get all columns that are nutrients
nutrient_columns = [col for col in df.columns if filter_nutrients(col)]

# Dictionary from the dataset to our Nutrient_ID
nutrient_columns

['energy_100g',
 'energy-from-fat_100g',
 'fat_100g',
 'saturated-fat_100g',
 'monounsaturated-fat_100g',
 'polyunsaturated-fat_100g',
 'omega-3-fat_100g',
 'omega-6-fat_100g',
 'omega-9-fat_100g',
 'trans-fat_100g',
 'cholesterol_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'starch_100g',
 'polyols_100g',
 'fiber_100g',
 'proteins_100g',
 'casein_100g',
 'serum-proteins_100g',
 'nucleotides_100g',
 'salt_100g',
 'sodium_100g',
 'alcohol_100g',
 'vitamin-a_100g',
 'beta-carotene_100g',
 'vitamin-d_100g',
 'vitamin-e_100g',
 'vitamin-k_100g',
 'vitamin-c_100g',
 'vitamin-b1_100g',
 'vitamin-b2_100g',
 'vitamin-pp_100g',
 'vitamin-b6_100g',
 'vitamin-b9_100g',
 'folates_100g',
 'vitamin-b12_100g',
 'biotin_100g',
 'pantothenic-acid_100g',
 'silica_100g',
 'bicarbonate_100g',
 'potassium_100g',
 'chloride_100g',
 'calcium_100g',
 'phosphorus_100g',
 'iron_100g',
 'magnesium_100g',
 'zinc_100g',
 'copper_100g',
 'manganese_100g',
 'fluoride_100g',
 'selenium_100g',
 'chromium_100g',
 'mo

In [4]:
nutrient_mapping = {
    # "energy_100g": 30,
    "fat_100g": 54,
    "saturated-fat_100g": 33,
    "monounsaturated-fat_100g": 31,
    "polyunsaturated-fat_100g": 32,
    "omega-3-fat_100g": 32,
    "omega-6-fat_100g": 32,
    "omega-9-fat_100g": 31,
    "cholesterol_100g": 26,
    "carbohydrates_100g": 23,
    "sugars_100g": 51,
    "fiber_100g": 34,
    "proteins_100g": 46,
    "salt_100g": 50,
    "sodium_100g": 50,
    "alcohol_100g": 20,
    "vitamin-a_100g": 55,
    "beta-carotene_100g": 25,
    "vitamin-d_100g": 60,
    "vitamin-e_100g": 61,
    "vitamin-k_100g": 63,
    "vitamin-c_100g": 59,
    "vitamin-b1_100g": 53,
    "vitamin-b2_100g": 48,
    "vitamin-pp_100g": 43,
    "vitamin-b6_100g": 58,
    "vitamin-b9_100g": 37,
    "folates_100g": 37,
    "vitamin-b12_100g": 56,
    "potassium_100g": 45,
    "calcium_100g": 22,
    "phosphorus_100g": 44,
    "iron_100g": 39,
    "magnesium_100g": 42,
    "zinc_100g": 65,
    "copper_100g": 28,
    "selenium_100g": 49,
    "caffeine_100g": 21
}

In [5]:
# Fill NaN values with 0
df.fillna(0, inplace=True)

C:\Users\Adama\AppData\Local\Temp\ipykernel_19364\2475422004.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


In [6]:
# Read the Nutrients CSV file and parse it properly
nutrients = pd.read_csv("../Data/CSV/Nutrients.csv")
nutrients.rename(columns={nutrients.columns[0]: 'ID'}, inplace=True)
nutrients

,ID,Name,Unit
0,1,10:0,g
1,2,12:0,g
2,3,14:0,g
3,4,16:0,g
4,5,16:1,g
...,...,...,...
60,61,Vitamin E (alpha-tocopherol),mg
61,62,"Vitamin E, added",mg
62,63,Vitamin K (phylloquinone),mcg
63,64,Water,g


In [7]:
# It seems that all sizes are in grams
# We need to convert them to the unit found in the above CSV
to_unit = {
    "kcal": 1,
    "g": 1,
    "mg": 1e-3,
    "mcg": 1e-6
}

for nutrient, nutrient_id in nutrient_mapping.items():
    df[nutrient] = df[nutrient] / to_unit[nutrients.loc[nutrient_id - 1, "Unit"]]

In [8]:
df

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,14.0,0.0,0.0
2,0000000004559,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0000000016087,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.0
7,0000000016124,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489055711,2017-03-09T10:35:11Z,1489055712,2017-03-09T10:35:12Z,Organic Muesli,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0
12,0000000016872,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489055650,2017-03-09T10:34:10Z,1489055651,2017-03-09T10:34:11Z,Zen Party Mix,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355884,95659305,http://world-en.openfoodfacts.org/product/9565...,usda-ndb-import,1489052093,2017-03-09T09:34:53Z,1489052094,2017-03-09T09:34:54Z,Dal Makhani,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0
355888,9577283844672,http://world-en.openfoodfacts.org/product/9577...,openfoodfacts-contributors,1473945917,2016-09-15T13:25:17Z,1487262680,2017-02-16T16:31:20Z,100% Pur Jus 4 agrumes,Jus 4 agrumes,1 L,...,100.0,0.0,0.0,0.0,0.0,0.0,4.0,-3.0,0.0,0.0
355968,9780803738782,http://world-en.openfoodfacts.org/product/9780...,usda-ndb-import,1489069944,2017-03-09T14:32:24Z,1489069945,2017-03-09T14:32:25Z,Organic Z Bar,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,11.0,0.0,0.0
355985,9787461062105,http://world-en.openfoodfacts.org/product/9787...,usda-ndb-import,1489092300,2017-03-09T20:45:00Z,1489092300,2017-03-09T20:45:00Z,Natural Cassava,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0


## Export Parsed Dataset

In [9]:
# Export the filtered DataFrame as CSV
df.to_csv("../Data/clean_openfoodfacts.csv", index=False)

# Learn Importance Values per Nutrient

In [10]:
X = df[nutrient_mapping.keys()]
y = df[score_col]

# Range for nutrition score
ymin, ymax = y.min(), y.max()
ymin, ymax

(np.float64(-15.0), np.float64(40.0))

In [11]:
# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
model = OLS(y_train, X_train)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                   
============================================================================================
Dep. Variable:     nutrition-score-uk_100g   R-squared (uncentered):                   0.843
Model:                                 OLS   Adj. R-squared (uncentered):              0.843
Method:                      Least Squares   F-statistic:                          2.083e+04
Date:                     Tue, 14 Jan 2025   Prob (F-statistic):                        0.00
Time:                             15:30:25   Log-Likelihood:                     -4.3719e+05
No. Observations:                   143437   AIC:                                  8.745e+05
Df Residuals:                       143400   BIC:                                  8.748e+05
Df Model:                               37                                                  
Covariance Type:                 nonrobust                                                  
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
fat_100g                     0.1937      0.001    143.185      0.000       0.191       0.196
saturated-fat_100g           0.4703      0.003    162.372      0.000       0.465       0.476
monounsaturated-fat_100g    -0.0496      0.005    -10.989      0.000      -0.058      -0.041
polyunsaturated-fat_100g     0.0197      0.006      3.361      0.001       0.008       0.031
omega-3-fat_100g            -0.2628      0.070     -3.753      0.000      -0.400      -0.126
omega-6-fat_100g            -0.0173      0.030     -0.578      0.563      -0.076       0.041
omega-9-fat_100g            -0.0093      0.061     -0.153      0.878      -0.128       0.109
cholesterol_100g             0.0002   3.78e-05      5.785      0.000       0.000       0.000
carbohydrates_100g           0.0226      0.001     41.960      0.000       0.022       0.024
sugars_100g                  0.2004      0.001    218.972      0.000       0.199       0.202
fiber_100g                  -0.2096      0.001   -195.944      0.000      -0.212      -0.207
proteins_100g                0.1365      0.002     81.685      0.000       0.133       0.140
salt_100g                   -0.0298      0.017     -1.774      0.076      -0.063       0.003
sodium_100g                  0.0758      0.043      1.775      0.076      -0.008       0.160
alcohol_100g                 0.3840      0.250      1.534      0.125      -0.107       0.875
vitamin-a_100g               0.0002   2.18e-05     11.190      0.000       0.000       0.000
beta-carotene_100g       -1.056e-05   8.68e-06     -1.217      0.224   -2.76e-05    6.46e-06
vitamin-d_100g               0.0005      0.013      0.035      0.972      -0.026       0.027
vitamin-e_100g              -0.0013      0.001     -1.591      0.112      -0.003       0.000
vitamin-k_100g            1.228e-07   1.63e-07      0.752      0.452   -1.97e-07    4.43e-07
vitamin-c_100g           -3.155e-05   3.25e-05     -0.970      0.332   -9.53e-05    3.22e-05
vitamin-b1_100g             -0.0003   4.48e-05     -5.964      0.000      -0.000      -0.000
vitamin-b2_100g          -7.215e-05   7.63e-05     -0.946      0.344      -0.000    7.74e-05
vitamin-pp_100g              0.0001      0.000      0.664      0.507      -0.000       0.000
vitamin-b6_100g              0.0003   5.87e-05      5.016      0.000       0.000       0.000
vitamin-b9_100g          -2.858e-07   2.13e-07     -1.341      0.180   -7.04e-07    1.32e-07
folates_100g             -2.698e-08   2.41e-08     -1.119      0.263   -7.43e-08    2.03e-08
vitamin-b12_100g          -2.55e-05      0.000     -0.170      0.865      -0.000       0.000
potassium_100g              -0.0006   5.23e-05

In [13]:
# Testing the model
y_hat = model.predict(results.params, X_test)

In [14]:
# Calculate the mean squared error
se = (y_hat - y_test) ** 2
mse = se.mean()
mse

np.float64(34.05498317097483)

# Update Importance Values in the Nutrients File

In [15]:
# Create a dataframe with the importance weight of each nutrient
params = results.params
params = pd.DataFrame(
    {
        "Name": params.index,
        "ID": params.index.map(nutrient_mapping),
        "Importance": params.values
    }
).sort_values("Importance", ascending=False)
params

,Name,ID,Importance
1,saturated-fat_100g,33,4.703437e-01
14,alcohol_100g,20,3.839889e-01
9,sugars_100g,51,2.003602e-01
0,fat_100g,54,1.936718e-01
11,proteins_100g,46,1.364768e-01
33,zinc_100g,65,9.214362e-02
13,sodium_100g,50,7.580903e-02
8,carbohydrates_100g,23,2.263739e-02
3,polyunsaturated-fat_100g,32,1.974039e-02
17,vitamin-d_100g,60,4.687460e-04


In [16]:
# Sum importance values per nutrient ID
params = params.groupby("ID").sum().reset_index()
params

,ID,Name,Importance
0,20,alcohol_100g,3.839889e-01
1,21,caffeine_100g,-8.014898e-05
2,22,calcium_100g,9.350242e-05
3,23,carbohydrates_100g,2.263739e-02
4,25,beta-carotene_100g,-1.056331e-05
5,26,cholesterol_100g,2.184793e-04
6,28,copper_100g,-1.249696e-03
7,31,omega-9-fat_100gmonounsaturated-fat_100g,-5.882083e-02
8,32,polyunsaturated-fat_100gomega-6-fat_100gomega-...,-2.604294e-01
9,33,saturated-fat_100g,4.703437e-01


In [17]:
# Export the DataFrame to a CSV file
params.to_csv(
    "../Data/CSV/Nutrient_Importance.csv",
    index=False
)

In [18]:
# Join the importance values with the Nutrients DataFrame
nutrients = nutrients.merge(params, on="ID", how="left")
# Nutrients that are not in the model have an importance of 0
nutrients["Importance"].fillna(0, inplace=True)
nutrients

C:\Users\Adama\AppData\Local\Temp\ipykernel_19364\131524038.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nutrients["Importance"].fillna(0, inplace=True)


,ID,Name_x,Unit,Name_y,Importance
0,1,10:0,g,NaN,0.000000e+00
1,2,12:0,g,NaN,0.000000e+00
2,3,14:0,g,NaN,0.000000e+00
3,4,16:0,g,NaN,0.000000e+00
4,5,16:1,g,NaN,0.000000e+00
...,...,...,...,...,...
60,61,Vitamin E (alpha-tocopherol),mg,vitamin-e_100g,-1.297095e-03
61,62,"Vitamin E, added",mg,NaN,0.000000e+00
62,63,Vitamin K (phylloquinone),mcg,vitamin-k_100g,1.227973e-07
63,64,Water,g,NaN,0.000000e+00


In [19]:
# Export the DataFrame to a CSV file
nutrients.to_csv(
    "../Data/CSV/Nutrients.csv",
    index=False
)

# Calculate Score per Food / Beverage

In [20]:
# Load Food_Beverages CSV file
food_beverages = pd.read_csv("../Data/CSV/Food_Beverages.csv")
food_beverages

,ID,Description,Extra_Description,WWEIA_ID
0,11000000,"Milk, human",NaN,9602
1,11100000,"Milk, NFS",NaN,1004
2,11111000,"Milk, whole",leche fresca,1002
3,11112110,"Milk, reduced fat (2%)",NaN,1004
4,11112210,"Milk, low fat (1%)",NaN,1006
...,...,...,...,...
5427,99997810,Vegetables as ingredient in curry,NaN,9999
5428,99997815,Vegetables as ingredient in soups,NaN,9999
5429,99997820,Vegetables as ingredient in stews,NaN,9999
5430,99998130,Sauce as ingredient in hamburgers,NaN,9999


In [21]:
# Load FB_Values CSV file
fb_values = pd.read_csv("../Data/CSV/FB_Values.csv")
fb_values

,FB_ID,Nutrient_ID,Value
0,11100000,1,0.056
1,11111000,1,0.084
2,11112110,1,0.049
3,11112210,1,0.023
4,11113000,1,0.002
...,...,...,...
353010,99997810,65,0.280
353011,99997815,65,0.360
353012,99997820,65,0.340
353013,99998130,65,0.210


In [22]:
# Merge FB_Values and Nutrients
values_nutrients = fb_values.merge(nutrients, left_on="Nutrient_ID", right_on="ID")
values_nutrients

,FB_ID,Nutrient_ID,Value,ID,Name_x,Unit,Name_y,Importance
0,11100000,1,0.056,1,10:0,g,NaN,0.000000
1,11111000,1,0.084,1,10:0,g,NaN,0.000000
2,11112110,1,0.049,1,10:0,g,NaN,0.000000
3,11112210,1,0.023,1,10:0,g,NaN,0.000000
4,11113000,1,0.002,1,10:0,g,NaN,0.000000
...,...,...,...,...,...,...,...,...
353010,99997810,65,0.280,65,Zinc,mg,zinc_100g,0.092144
353011,99997815,65,0.360,65,Zinc,mg,zinc_100g,0.092144
353012,99997820,65,0.340,65,Zinc,mg,zinc_100g,0.092144
353013,99998130,65,0.210,65,Zinc,mg,zinc_100g,0.092144


In [23]:
# Calculate the score for each nutrient
values_nutrients["Score"] = values_nutrients["Value"] * values_nutrients["Importance"]
values_nutrients

,FB_ID,Nutrient_ID,Value,ID,Name_x,Unit,Name_y,Importance,Score
0,11100000,1,0.056,1,10:0,g,NaN,0.000000,0.000000
1,11111000,1,0.084,1,10:0,g,NaN,0.000000,0.000000
2,11112110,1,0.049,1,10:0,g,NaN,0.000000,0.000000
3,11112210,1,0.023,1,10:0,g,NaN,0.000000,0.000000
4,11113000,1,0.002,1,10:0,g,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
353010,99997810,65,0.280,65,Zinc,mg,zinc_100g,0.092144,0.025800
353011,99997815,65,0.360,65,Zinc,mg,zinc_100g,0.092144,0.033172
353012,99997820,65,0.340,65,Zinc,mg,zinc_100g,0.092144,0.031329
353013,99998130,65,0.210,65,Zinc,mg,zinc_100g,0.092144,0.019350


In [24]:
# Calculate the total score for each food/beverage
fb_score = values_nutrients.groupby("FB_ID")["Score"].sum().reset_index()

In [25]:
# Merge the two DataFrames and drop the duplicate FB_ID column
food_beverages = food_beverages.merge(fb_score, left_on="ID", right_on="FB_ID")
food_beverages.drop(columns=["FB_ID"], inplace=True)

In [26]:
# Clamp the values which are too extreme
food_beverages["Score"] = food_beverages["Score"].clip(lower=ymin, upper=ymax)
# Normalise the score between 0 and 1
food_beverages["Score"] = (food_beverages["Score"] - ymin) / (ymax - ymin)
# Score is originally calculated as 'unhealthy' = ymax, 'healthy' = ymin
# We want a nutrition score between 0 and 1, where 0 is unhealthy and 1 is healthy
food_beverages["Score"] = 1.0 - food_beverages["Score"]
# Make sure the score is normalised between 0 and 1
food_beverages["Score"] -= food_beverages["Score"].min()
food_beverages["Score"] /= food_beverages["Score"].max()
food_beverages

,ID,Description,Extra_Description,WWEIA_ID,Score
0,11100000,"Milk, NFS",NaN,1004,0.797943
1,11111000,"Milk, whole",leche fresca,1002,0.789158
2,11112110,"Milk, reduced fat (2%)",NaN,1004,0.800031
3,11112210,"Milk, low fat (1%)",NaN,1006,0.809064
4,11113000,"Milk, fat free (skim)",NaN,1008,0.815431
...,...,...,...,...,...
5426,99997810,Vegetables as ingredient in curry,NaN,9999,0.859519
5427,99997815,Vegetables as ingredient in soups,NaN,9999,0.867984
5428,99997820,Vegetables as ingredient in stews,NaN,9999,0.846245
5429,99998130,Sauce as ingredient in hamburgers,NaN,9999,0.000000


In [27]:
# Save the DataFrame to a CSV file
food_beverages.to_csv(
    "../Data/CSV/Food_Beverages.csv",
    index=False
)